In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/IoT Test project 2')

In [52]:
import os
import numpy as np
import torch
from PIL import Image

class PennFundanDataset(torch.utils.data.Dataset):
  def __init__(self,root,transforms):
      self.root= root
      self.transforms = transforms

      self.imgs = list(sorted(os.listdir("/content/drive/MyDrive/Colab Notebooks/IoT Test project 2/PNGImages")))
      self.masks = list(sorted(os.listdir("/content/drive/MyDrive/Colab Notebooks/IoT Test project 2/PedMasks")))


  def __getitem__(self,idx):
    img_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/IoT Test project 2/PNGImages",self.imgs[idx])
    mask_path = os.path.join("/content/drive/MyDrive/Colab Notebooks/IoT Test project 2/PedMasks",self.masks[idx])

    img = Image.open(img_path).convert("RGB")

    mask = Image.open(mask_path)

    mask = np.array(mask)

    obj_ids = np.unique(mask)

    obj_ids = obj_ids[1:]

    masks = mask == obj_ids[:,None,None]

    num_objs = len(obj_ids)

    boxes = []

    for i in range(num_objs):
      pos = np.where(masks[i])
      xmin = np.min(pos[1])
      xmax = np.max(pos[1])
      ymin = np.min(pos[0])
      ymax = np.max(pos[0])

      boxes.append([xmin,ymin,xmax,ymax])

    boxes = torch.as_tensor(boxes,dtype = torch.float32)

    labels = torch.ones((num_objs,),dtype=torch.int64)
    masks = torch.as_tensor(masks,dtype=torch.uint8)

    image_id = torch.tensor([idx])
    area = (boxes[:,3]-boxes[:,1])*(boxes[:,2]-boxes[:,0])

    iscrowd = torch.zeros((num_objs,),dtype=torch.int64)

    target = {}

    target["boxes"] = boxes
    target["labels"] = labels
    target["masks"] = masks
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd

    if self.transforms is not None:
      img, target = self.transforms(img, target)

    return img,target

  def __len__(self):
    return len(self.imgs)







In [38]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes =2 

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [39]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32,64,128,256,512),),aspect_ratios=((0.5,1.0,2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],output_size=7,sampling_ratio=2)

model = FasterRCNN(backbone,num_classes=2,rpn_anchor_generator=anchor_generator,box_roi_pool=roi_pooler)



In [40]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):

  model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

  in_features = model.roi_heads.box_predictor.cls_score.in_features

  model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)

  in_feaures_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
  hidden_layer = 256

  model.roi_heads.mask_predictor = MaskRCNNPredictor(in_feaures_mask,hidden_layer,num_classes)

  return model





In [41]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/IoT Test project 2/visionmain/references/detection')



In [42]:
pip install torch-util

In [43]:
from torchvision import transforms as T
def get_transforms(train):
  transforms = []
  transforms.append(torchvision.transforms.ToTensor())
  if train:
    transforms.append(T.RandomHorizontalFlip(0.5))
  return torchvision.transforms.Compose(transforms)

In [44]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

def collate_fn(batch):
    data_list, label_list = [], []
    for _data, _label in batch:
        data_list.append(_data)
        label_list.append(_label)
    return torch.Tensor(data_list), torch.LongTensor(label_list)

In [53]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFundanDataset('PennFundanDataset',get_transforms(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)   

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored